# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

---

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»).

🎯 _Нужно построить модель для задачи классификации, которая выберет подходящий тариф._

Предобработка данных не понадобится — мы её уже сделали.

Построим модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке самостоятельно.

## 1. Откроем и изучим файл датасета

In [50]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.metrics import accuracy_score

In [2]:
users_behavior = pd.read_csv('users_behavior.csv')

In [3]:
users_behavior.shape

(3214, 5)

In [4]:
users_behavior.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## 2. Разделите исходные данные на обучающую, валидационную и тестовую выборки

In [16]:
np.random.seed(42)

# 60% отводим под обучающую выборку
df_train, df_valid = train_test_split(users_behavior, test_size=0.4)
# половину из оставшихся 40% отдаем на валидационную, другую половину – на тестовую
df_valid, df_test = train_test_split(df_valid, test_size=0.5)

In [17]:
df_train

,calls,minutes,messages,mb_used,is_ultra
2369,62.0,436.40,89.0,8495.06,1
2234,70.0,489.70,13.0,10671.53,0
1058,64.0,423.95,44.0,7826.03,0
118,28.0,215.41,0.0,6045.15,0
1024,100.0,709.40,17.0,16964.33,0
...,...,...,...,...,...
1095,62.0,454.02,35.0,15018.46,0
1130,69.0,465.96,12.0,14982.27,0
1294,40.0,280.44,2.0,13934.54,0
860,72.0,410.23,68.0,16006.55,0


In [23]:
X_train, y_train = df_train.drop(columns=['is_ultra']), df_train['is_ultra']
X_train

In [25]:
X_test, y_test = df_test.drop(columns=['is_ultra']), df_test['is_ultra']
X_test

,calls,minutes,messages,mb_used
139,106.0,839.31,0.0,13269.80
1084,94.0,615.52,39.0,12728.96
787,34.0,263.06,73.0,15661.73
2024,27.0,147.66,39.0,7545.04
1253,62.0,446.89,7.0,21571.89
...,...,...,...,...
654,53.0,478.18,78.0,20152.53
1426,184.0,1225.70,135.0,24756.40
714,60.0,376.33,1.0,13479.08
2912,11.0,55.91,25.0,15217.06


In [26]:
X_valid, y_valid = df_valid.drop(columns=['is_ultra']), df_valid['is_ultra']
X_valid

,calls,minutes,messages,mb_used
2917,47.0,350.90,19.0,13867.87
221,98.0,729.83,105.0,18675.65
2787,67.0,503.17,25.0,13295.63
772,107.0,666.29,35.0,27935.42
1741,101.0,741.35,21.0,17014.08
...,...,...,...,...
869,69.0,482.18,40.0,11178.07
1351,0.0,0.00,9.0,2923.50
1844,86.0,630.51,54.0,14496.74
2579,35.0,268.55,71.0,23313.86


## 3. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

### logreg

In [53]:
logreg = LogisticRegression()

In [54]:
parameters = {'C': [0.1, 0.5, 1.0, 5, 10],
              'max_iter': range(50, 550, 100),
              'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

In [55]:
logreg = GridSearchCV(logreg, parameters, n_jobs=-1)

In [56]:
logreg.fit(X_train, y_train)

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.1, 0.5, 1.0, 5, 10],
                         'max_iter': range(50, 550, 100),
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [78]:
logreg.best_params_

{'C': 0.1, 'max_iter': 150, 'solver': 'newton-cg'}

In [77]:
accuracy_score(y_test, logreg.predict(X_test))

0.7480559875583204

### svc

In [71]:
svc = svm.SVC()

In [72]:
parameters = {'C': [0.1, 0.5, 1.0, 5, 10],
              'decision_function_shape' : ['ovo', 'ovr'],
              'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']}

In [73]:
svc = GridSearchCV(svc, parameters, n_jobs=-1)

In [74]:
svc.fit(X_train, y_train)

GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 0.5, 1.0, 5, 10],
                         'decision_function_shape': ['ovo', 'ovr'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [76]:
svc.best_score_

0.7417051342439944

In [81]:
svc.best_params_

{'C': 5, 'decision_function_shape': 'ovo', 'kernel': 'linear'}

In [88]:
accuracy_score(y_test, svc.predict(X_test))

0.7216174183514774

## DecisionTreeClassifier

In [83]:
tree = DecisionTreeClassifier()

In [84]:
parameters = {'criterion': ["gini", "entropy"],
              'max_depth': range(5, 25, 2)}

In [85]:
tree = GridSearchCV(tree, parameters, n_jobs=-1)

In [86]:
tree.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(5, 25, 2)})

In [87]:
tree.best_params_

{'criterion': 'entropy', 'max_depth': 7}

In [104]:
tree.best_score_

0.8112038220846511

In [89]:
accuracy_score(y_test, tree.predict(X_test))

0.776049766718507

## Random Forest

In [99]:
tree = RandomForestClassifier()

In [100]:
parameters = {'criterion': ["gini", "entropy"],
              'max_depth': range(2, 15, 1),
              'n_estimators' : range(10, 200, 20)}

In [101]:
tree = GridSearchCV(tree, parameters, n_jobs=-1)

In [102]:
tree.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 15),
                         'n_estimators': range(10, 200, 20)})

In [105]:
tree.best_params_

{'criterion': 'gini', 'max_depth': 9, 'n_estimators': 150}

In [106]:
tree.best_score_

0.8112038220846511

In [95]:
accuracy_score(y_test, tree.predict(X_test))

0.8055987558320373

### random forest лучше всех

## Проверьте качество модели на тестовой выборке

In [108]:
accuracy_score(y_valid, tree.predict(X_valid))

0.8195956454121306

не переобучились

## Проверьте модели на вменяемость